### 네이버쇼핑에서 카테고리 클롤링

In [345]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from nltk import word_tokenize
from bs4 import BeautifulSoup
import urllib.request
import urllib.parse
import math
import threading
from selenium import webdriver
from time import sleep
from konlpy.tag import Okt

In [2]:
df = pd.read_csv("items.csv")
df.columns =['seq','code','img','name','price','seller','detail']

In [393]:
df.tail()

,seq,code,img,name,price,seller,detail
82971,"49,381",1000236324,http://crunchprice.com/goods/goods_view.php?go...,[아펙슨] 아펙슨 AT-533/3in1/멀티카타/건타카/타카총,"12,000원",공구랜드,./goods_register.php?goodsNo=1000236324
82972,"49,380",1000236323,http://crunchprice.com/goods/goods_view.php?go...,[라이카] 라이카 DISTO D1/최대40m/레이저 거리측정기,"85,000원",공구랜드,./goods_register.php?goodsNo=1000236323
82973,"49,379",1000236322,http://crunchprice.com/goods/goods_view.php?go...,[아펙슨] 아펙슨 AT-569/4in1/멀티카타/건타카/타카총,"18,800원",공구랜드,./goods_register.php?goodsNo=1000236322
82974,"49,378",1000236321,http://crunchprice.com/goods/goods_view.php?go...,한국OSG 엔드밀/앤드밀/2날 4날/2mm~14mm,"6,140원",공구랜드,./goods_register.php?goodsNo=1000236321
82975,"49,377",1000236320,http://crunchprice.com/goods/goods_view.php?go...,태진 국산 EPB64C 전자부품보관대,"739,200원",공구랜드,./goods_register.php?goodsNo=1000236320


### 상품명을 token으로 분리하여 단어 추출

In [30]:
res = []
twitter = Okt()
for txt in list(df['name'].values):
    tokens = word_tokenize(txt)
    nonus = twitter.nouns(txt) 
    
    tokens = [w for w in tokens if w not in ['/','[',']','(',')','x',',','-','%','#']]
    tokens.extend(nonus)
 
    res.append(set(tokens)) 

# 한글자 토큰은 제외한다
fres = [[w for w in tokens if len(w) != 1] for tokens in res]

82976

### 카테고리 크롤링 시작

In [342]:
def get_cate(tc):
    count = {}
    for cate in tc:
        if cate in count.keys():
            continue
        else:
            count[cate] = tc.count(cate)
            
    count = pd.DataFrame(list(count.items()))
    count = count.sort_values(by=1, ascending=False).reset_index(drop=True)
    
    # 많이 출현한 카테고리 10개 추출
    cten = count.iloc[:10]
    # softmax로 비율 계산
    e = np.exp(cten[1])
    cten[2] = (e / np.sum(e)).round(3)

    # 상위 3개 카테고리
    t3c = cten.iloc[:3][0].values
    
    
    # 카테고리를 구성하는 단어로 빈도 계산
    tmp = {}
    for idx, tokens in enumerate(cten[0].values):
        ws = cten[0].values[idx].replace(' ','').split('>')

        for w in ws:
            if w in tmp.keys():
                tmp[w] += cten.iloc[idx][1]
            else:
                tmp[w] = cten.iloc[idx][1]
                
    e = np.exp(list(tmp.values()))
    tmp2 = pd.DataFrame( e/ np.sum(e), list(tmp.keys()))
    tmp2[0] = tmp2[0].round(5)
    
    t3w = tmp2.iloc[:3].index.values
    
    return t3c, t3w

In [ ]:
driver = webdriver.Chrome()
 
total_res = []
for idx, tokens in enumerate(fres):
    tc = []
    for w in tokens:
        try:
            url = "https://search.shopping.naver.com/search/all.nhn?query={}".format(w)
            driver.get(url)
            sleep(0.5)
 
            soup = BeautifulSoup(driver.find_element_by_tag_name('body').get_attribute('innerHTML')   , 'html.parser')
 
            cates = driver.find_elements_by_css_selector("span.depth")
            for c in cates:
                tc.append(c.text)
        except Exception as e:
            print(e)
    
    c3, w3 = get_cate(tc) 
    total_res.append([df.iloc[idx]['seq'], c3, w3])
    
driver.close()

In [350]:
c_res = pd.DataFrame(total_res)
c_res = c_res.rename(columns={0:'seq',1:'cate',2:'keyword'})

### 상품명과 카테고리 비교

In [374]:
merged = df.merge(c_res, on=["seq"])
ex10 = merged.filter(items=['seq','name','cate','keyword'])[:10]

In [400]:
for idx in range(len(ex10)):
    print("name : ",ex10.iloc[idx]['name'])
    print("category:")
    for c in ex10.iloc[idx]['cate']:
        print("    ",c)
    print("keyword : ",",".join(ex10.iloc[idx]['keyword']))
    print("========================================================")

name :  이테크 E400 / E-400 5W 보급형 고성능 업무용 무전기
category:
     디지털/가전 > 생활가전 > 무전기 > 업무용무전기
     생활/건강 > 생활용품 > 세제/세정제 > 세탁세제
     식품 > 건강식품 > 영양제 > 기타건강기능식품
keyword :  디지털/가전,생활가전,무전기
name :  라디오텍 생활무전기 RT320S RT-320S 매장 음식점 병원 식당
category:
     디지털/가전 > 생활가전 > 무전기 > 생활용무전기
     디지털/가전 > 생활가전 > 무전기 > 업무용무전기
     생활/건강 > 주방용품 > 칼/커팅기구 > 다지기
keyword :  디지털/가전,생활가전,무전기
name :  [코카콜라]미닛메이드 망고 175mL x 30캔[도매퍼스트]
category:
     식품 > 음료 > 청량/탄산음료 > 콜라
     식품 > 농산물 > 과일 > 망고
     식품 > 음료 > 주스/과즙음료 > 오렌지주스
keyword :  식품,음료,청량/탄산음료
name :  [모토로라] 생활무전기 TLKR-T82EX 익스트림 히든디스플레이/27채널/이어마이크/휴대용케이스
category:
     디지털/가전 > 생활가전 > 무전기 > 생활용무전기
     디지털/가전 > 휴대폰액세서리 > 휴대폰케이스 > 기타케이스
     패션잡화 > 남성신발 > 스니커즈
keyword :  디지털/가전,생활가전,무전기
name :  오뚜기 통 흑 후추 50G 1박스 12개
category:
     생활/건강 > 공구 > 포장용품 > 택배박스
     식품 > 가공식품 > 조미료 > 후추
     식품 > 가공식품 > 간편조리식
keyword :  생활/건강,공구,포장용품
name :  [아재스토리](영상상품)텐미닛 바르는 제모크림(Body hair removal) 100ml/다리털제모/겨드랑이털제모/배렛나루제모/왁싱
category:
     화장품/미용 > 바디케어 > 제모제
     화장품/미용 > 

1. konlpy의 twitter tokennizer를 사용하여 상품명에서 명사만 추출
2. 추출한 명사를 네이버 쇼핑에 검색하여 검색결과 목록에서 카테고리 정보만 크롤링
3. 크롤링한 카테고리중 출현빈도가 높은 3개의 카테고리 추출하여 저장
4. 크롤링한 카테고리중 출현빈도가 높은 10개의 카테고리를 토큰으로 분리하여 출현빈도가 높은 3개를 키워드로 저장